In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import numpy as np


In [2]:
url = "https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow"
response = requests.get(url)
resp = json.loads(response.text)

In [3]:
df = pd.DataFrame(resp['items'])

df['creation_date'] = df['creation_date'].apply(lambda x: datetime.fromtimestamp(x))
df['last_activity_date'] = df['last_activity_date'].apply(lambda x: datetime.fromtimestamp(x))
df['closed_date'] = df['closed_date'].apply(lambda x: datetime.fromtimestamp(x) if pd.notna(x) else None)
df['protected_date'] = df['protected_date'].apply(lambda x: datetime.fromtimestamp(x) if pd.notna(x) else None)

df.head(2)

,tags,owner,is_answered,view_count,answer_count,score,last_activity_date,creation_date,question_id,content_license,link,title,last_edit_date,accepted_answer_id,closed_date,closed_reason,protected_date
0,"[perl, imagemagick, perlmagick]","{'reputation': 665, 'user_id': 2744953, 'user_...",False,25,1,0,2022-05-04 23:15:10,2022-05-04 16:48:49,72119828,CC BY-SA 4.0,https://stackoverflow.com/questions/72119828/p...,Perl ImageMagick: how to disable printing of c...,NaN,NaN,NaT,NaN,NaT
1,"[perl, testing, read-eval-print-loop, test-more]","{'reputation': 21, 'user_id': 11199751, 'user_...",False,24,0,0,2022-05-04 21:22:20,2022-05-04 20:44:29,72121076,CC BY-SA 4.0,https://stackoverflow.com/questions/72121076/e...,execute a programman REPL with perl,1.651717e+09,NaN,NaT,NaN,NaT


#### Obtener el número de respuestas contestadas y no contestadas

In [ ]:
answered_count = df[df['is_answered']==True]['is_answered'].shape[0]
unanswered_count = df[df['is_answered']==False]['is_answered'].shape[0]

print(f'Total de preguntas contestadas: {answered_count}')
print(f'Total de preguntas no contestadas: {unanswered_count}')

In [8]:
answeres = [x['is_answered'] for x in resp['items']]
answered_count = sum(answeres)
not_answered_count = len(answeres) - answered_count

print(f'Total de preguntas contestadas: {answered_count}')
print(f'Total de preguntas no contestadas: {not_answered_count}')

Total de preguntas contestadas: 25
Total de preguntas no contestadas: 5


#### Obtener la respuesta con menor número de vistas

In [6]:
xx = json.loads(df[df['view_count'] == df['view_count'].min()].to_json(orient='records'))
print(xx)

[{'tags': ['perl', 'testing', 'read-eval-print-loop', 'test-more'], 'owner': {'reputation': 21, 'user_id': 11199751, 'user_type': 'registered', 'profile_image': 'https://lh6.googleusercontent.com/-v6oOEjABbUY/AAAAAAAAAAI/AAAAAAAAAAA/ACHi3re9pT0Wzi9MhE2ocnsB65OvmYCjEQ/mo/photo.jpg?sz=256', 'display_name': 'Guilherme Gi&#225;como Simoes', 'link': 'https://stackoverflow.com/users/11199751/guilherme-gi%c3%a1como-simoes'}, 'is_answered': False, 'view_count': 24, 'answer_count': 0, 'score': 0, 'last_activity_date': 1651699340000, 'creation_date': 1651697069000, 'question_id': 72121076, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/72121076/execute-a-programman-repl-with-perl', 'title': 'execute a programman REPL with perl', 'last_edit_date': 1651717340.0, 'accepted_answer_id': None, 'closed_date': None, 'closed_reason': None, 'protected_date': None}]


#### Obtener la respuesta más vieja y más actual

In [9]:
df['creation_date'] = df['creation_date'].apply(lambda x: datetime.fromtimestamp(x))
df['last_activity_date'] = df['last_activity_date'].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
old = df[df['creation_date'] == df['creation_date'].min()].to_json()
new = df[df['creation_date'] == df['creation_date'].max()].to_json()

print(f'respuesta mas antigua: {old} \n')
print(f'respuesta mas reciente: {new}')

#### Obtener la respuesta del owner que tenga una mayor reputación

In [10]:
def aux(row):
    try:
        return row['reputation']
    except KeyError:
        return 0

In [12]:
df_1 = df.copy()
df_1['owner_reputation'] = df_1['owner'].apply(aux)
max_owner_reputation_answer = df_1[df_1['owner_reputation'] == df_1['owner_reputation'].max()].to_json()
print(f'maximo reputacion de una respuesta: {max_owner_reputation_answer}')


maximo reputacion de una respuesta: {"tags":{"25":["perl","subroutine"]},"owner":{"25":{"reputation":89206,"user_id":31610,"user_type":"registered","accept_rate":87,"profile_image":"https:\/\/www.gravatar.com\/avatar\/20571f32acd27d2cf5f1b1f42ea1a75b?s=256&d=identicon&r=PG","display_name":"Geo","link":"https:\/\/stackoverflow.com\/users\/31610\/geo"}},"is_answered":{"25":true},"view_count":{"25":1509},"answer_count":{"25":5},"score":{"25":14},"last_activity_date":{"25":1651425926},"creation_date":{"25":1251840375},"last_edit_date":{"25":1651425926.0},"question_id":{"25":1364734},"content_license":{"25":"CC BY-SA 4.0"},"link":{"25":"https:\/\/stackoverflow.com\/questions\/1364734\/can-perl-method-calls-be-intercepted"},"title":{"25":"Can Perl method calls be intercepted?"},"accepted_answer_id":{"25":1364905.0},"closed_date":{"25":null},"closed_reason":{"25":null},"protected_date":{"25":null},"owner_reputation":{"25":89206}}


In [6]:
df["owner"]

0     {'reputation': 21, 'user_id': 11199751, 'user_...
1     {'reputation': 29, 'user_id': 10563266, 'user_...
2     {'reputation': 665, 'user_id': 2744953, 'user_...
3     {'reputation': 14188, 'user_id': 269126, 'user...
4     {'user_type': 'does_not_exist', 'display_name'...
5     {'reputation': 11, 'user_id': 17860614, 'user_...
6     {'reputation': 33, 'user_id': 7602733, 'user_t...
7     {'reputation': 409, 'user_id': 5128398, 'user_...
8     {'reputation': 602, 'user_id': 1996150, 'user_...
9     {'reputation': 320, 'user_id': 7344596, 'user_...
10    {'reputation': 3, 'user_id': 19026977, 'user_t...
11    {'reputation': 16434, 'user_id': 450148, 'user...
12    {'reputation': 3613, 'user_id': 402649, 'user_...
13    {'reputation': 1352, 'user_id': 4208719, 'user...
14    {'reputation': 1000, 'user_id': 2993440, 'user...
15    {'reputation': 1, 'user_id': 19022401, 'user_t...
16    {'reputation': 529, 'user_id': 6368549, 'user_...
17    {'reputation': 11, 'user_id': 18835228, 'u

#### Imprimir en consola del punto 2 al 5

In [105]:
date = datetime.fromtimestamp(1651008069)
date.strftime('%Y-%m-%d %H:%M:%S')

'2022-04-26 16:21:09'